In [53]:
import datetime
import requests
import pandas as pd

In [54]:
companies = ['RELIANCE', 'BAJAJFINSV', 'HDFCBANK', 'HDFC', 'ITC', 'INFY', 'ADANIPORTS', 'ICICIBANK', 'BAJFINANCE', 'TCS', 'KOTAKBANK', 'LT', 'AXISBANK', 'MARUTI', 'SBIN', 'TATAMOTERS', 'TATASTEEL', 'HINDALCO', 'BHARTIARTL', 'NTPC', 'EICHERMOT', 'APOLLOHOSP', 'M&N', 'INDUSINDBK', 'ASIANPAINT', 'HINDUNILVR', 'ONGC', 'BAJAJ-AUTO', 'HEROMOTOCO', 'TITAN', 'WIPRO', 'HDFCLIFE', 'SHREECEM', 'ULTRACEMCO', 'TECHM', 'SUNPHARMA', 'CIPLA', 'HCLTECH', 'TATACONSUM', 'BPCL', 'UPL', 'SBILIFE', 'JSWSTEEL', 'POWERGRID', 'DIVISLAB', 'COALINDIA', 'GRASIM', 'DRREDDY', 'BRITANNIA', 'MESTLEIND']

In [55]:
# set up today and yesterday
current_date = datetime.date.today()
today = current_date.strftime("%d%m%Y")
# today = '14102022'


yesterday = current_date - datetime.timedelta(days=1)
yesterday = yesterday.strftime('%d%m%Y')
# yesterday = '13102022'


In [56]:
# setup the source url to fetch the new data from nseindia.com
source_file_name = 'sec_bhavdata_full_{}.csv'
base_url = r'https://archives.nseindia.com/products/content/'
source_url = base_url+source_file_name.format(yesterday)
source_url

'https://archives.nseindia.com/products/content/sec_bhavdata_full_13102022.csv'

SET UP THE MAIN DATA FILE

In [18]:
# setup the column names of the new file
column_names = ['Symbol', 'Series', 'Date', 'PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice', 'ClosePrice'+yesterday, 'AvgPrice', 'TotalTradedQuantity'+yesterday, 'TurnoverInLacs'+yesterday, 'NoOfTrades', 'DeliveryQuantity'+yesterday, 'DeliveryPercentage'+yesterday]
main_bhav_data = pd.read_csv(source_url, names=column_names)
main_bhav_data = main_bhav_data.drop(columns=['Date','PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice', 'AvgPrice', 'NoOfTrades'])
main_bhav_data = main_bhav_data.loc[main_bhav_data['Symbol'].isin(companies)]

In [19]:
main_bhav_data

,Symbol,Series,ClosePrice10102022,TotalTradedQuantity10102022,TurnoverInLacs10102022,DeliveryQuantity10102022,DeliveryPercentage10102022
54,ADANIPORTS,EQ,808.10,4642360,37440.41,997903,21.50
139,APOLLOHOSP,EQ,4340.60,207878,8994.10,38732,18.63
173,ASIANPAINT,EQ,3277.95,734377,24067.24,302946,41.25
205,AXISBANK,EQ,776.85,14106654,107996.41,8697279,61.65
219,BAJAJ-AUTO,EQ,3616.95,240975,8635.05,119562,49.62
222,BAJAJFINSV,EQ,1684.40,1229639,20620.98,313142,25.47
226,BAJFINANCE,EQ,7293.40,653113,47535.02,139108,21.30
284,BHARTIARTL,EQ,792.35,3519416,27722.40,2247568,63.86
311,BPCL,EQ,304.65,2365616,7192.66,824116,34.84
316,BRITANNIA,EQ,3797.85,185199,6981.10,108465,58.57


In [20]:
# Filtering the main data file.
company_count = {}
for company in companies:
    company_count[company] = 0
is_valid = [0 for _ in main_bhav_data.index]
main_bhav_data.insert(1, 'is_valid', is_valid)

for ind in main_bhav_data.index:
    company = main_bhav_data.at[ind, 'Symbol']
    if not company_count[company]:
        company_count[company] = 1
        main_bhav_data.at[ind, 'is_valid'] = 1

main_bhav_data = main_bhav_data.loc[main_bhav_data['is_valid'] == 1]

main_bhav_data.to_csv("nse_data_file.csv", index=False)

SET UP TEMPORARY DATA FILE AND MERGE THE REQUIRED DATA TO MAIN FILE

In [57]:
main_bhav_data = pd.read_csv('nse_data_file.csv')

In [58]:
main_bhav_data

,Symbol,is_valid,Series,ClosePrice10102022,ClosePrice11102022,ClosePrice12102022,ClosePrice13102022,TotalTradedQuantity10102022,TotalTradedQuantity11102022,TotalTradedQuantity12102022,...,TurnoverInLacs12102022,TurnoverInLacs13102022,DeliveryQuantity10102022,DeliveryQuantity11102022,DeliveryQuantity12102022,DeliveryQuantity13102022,DeliveryPercentage10102022,DeliveryPercentage11102022,DeliveryPercentage12102022,DeliveryPercentage13102022
0,ADANIPORTS,1,EQ,808.10,797.35,803.90,784.75,4642360,6193388,5628537,...,44927.98,59083.82,997903,1304344,940688,1510646,21.50,21.06,16.71,20.16
1,APOLLOHOSP,1,EQ,4340.60,4275.55,4287.05,4266.05,207878,258216,621343,...,26635.00,11822.66,38732,106718,414814,134839,18.63,41.33,66.76,48.67
2,ASIANPAINT,1,EQ,3277.95,3298.70,3248.20,3209.00,734377,1411316,1812549,...,58961.14,25075.25,302946,523181,730138,356092,41.25,37.07,40.28,45.66
3,AXISBANK,1,EQ,776.85,785.55,808.70,802.00,14106654,15953495,21228018,...,170060.88,104649.02,8697279,8916019,14015974,7174692,61.65,55.89,66.03,55.08
4,BAJAJ-AUTO,1,EQ,3616.95,3528.75,3624.50,3604.80,240975,258827,496229,...,17956.79,12176.42,119562,128336,146509,175361,49.62,49.58,29.52,52.10
5,BAJAJFINSV,1,EQ,1684.40,1672.65,1694.65,1681.75,1229639,1402150,1538819,...,25957.80,17876.84,313142,439030,477717,248944,25.47,31.31,31.04,23.37
6,BAJFINANCE,1,EQ,7293.40,7202.80,7226.20,7151.65,653113,732206,740731,...,53385.38,43572.21,139108,239519,221445,157860,21.30,32.71,29.90,25.94
7,BHARTIARTL,1,EQ,792.35,780.30,775.55,768.90,3519416,3699862,5984299,...,46400.86,21903.33,2247568,2047994,4005208,1519596,63.86,55.35,66.93,53.47
8,BPCL,1,EQ,304.65,299.75,302.50,300.30,2365616,2652669,7395855,...,22543.02,9128.44,824116,1425085,2450766,1334588,34.84,53.72,33.14,44.06
9,BRITANNIA,1,EQ,3797.85,3727.90,3763.00,3784.60,185199,202644,271156,...,10184.82,5919.31,108465,118939,199778,94118,58.57,58.69,73.68,60.04


In [59]:
source_url = base_url+source_file_name.format(today)
# setup the column names of the new file
column_names = ['Symbol', 'Series', 'Date', 'PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice', 'ClosePrice'+today, 'AvgPrice', 'TotalTradedQuantity'+today, 'TurnoverInLacs'+today, 'NoOfTrades', 'DeliveryQuantity'+today, 'DeliveryPercentage'+today]

# fetch today's bhav data file from nseindia.com
today_bhav_data = pd.read_csv(source_url, names=column_names)

today_bhav_data = today_bhav_data.drop(columns=['Date','PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice', 'AvgPrice', 'NoOfTrades'])

today_bhav_data = today_bhav_data.loc[today_bhav_data['Symbol'].isin(companies)]

company_count = {}
for company in companies:
    company_count[company] = 0

is_valid = [0 for _ in today_bhav_data.index]

today_bhav_data.insert(1, 'is_valid', is_valid)

for ind in today_bhav_data.index:
    company = today_bhav_data.at[ind, 'Symbol']
    if not company_count[company]:
        company_count[company] = 1
        today_bhav_data.at[ind, 'is_valid'] = 1

today_bhav_data = today_bhav_data.loc[today_bhav_data['is_valid'] == 1]

today_bhav_data.to_csv('temp_nse_data.csv', index=False)

In [60]:
today_bhav_data = pd.read_csv('temp_nse_data.csv')

In [61]:
today_bhav_data

,Symbol,is_valid,Series,ClosePrice14102022,TotalTradedQuantity14102022,TurnoverInLacs14102022,DeliveryQuantity14102022,DeliveryPercentage14102022
0,ADANIPORTS,1,EQ,780.10,4562526,36025.51,650375,14.25
1,APOLLOHOSP,1,EQ,4297.15,254366,10930.27,94375,37.10
2,ASIANPAINT,1,EQ,3185.50,845765,27163.17,511708,60.50
3,AXISBANK,1,EQ,800.50,8220645,66349.42,3810137,46.35
4,BAJAJ-AUTO,1,EQ,3570.50,499983,18069.55,256918,51.39
5,BAJAJFINSV,1,EQ,1688.30,1239168,21010.99,387881,31.30
6,BAJFINANCE,1,EQ,7272.50,839012,61029.68,267321,31.86
7,BHARTIARTL,1,EQ,765.95,3608210,27861.01,2176149,60.31
8,BPCL,1,EQ,297.85,2577730,7747.98,1166818,45.27
9,BRITANNIA,1,EQ,3796.25,350141,13294.52,253321,72.35


In [62]:
# setup the column names for yesterday's data from the main bhav data file.
header_main = ['ClosePrice'+yesterday, 'TotalTradedQuantity'+yesterday, 'TurnoverInLacs'+yesterday, 'DeliveryQuantity'+yesterday, 'DeliveryPercentage'+yesterday]
header_main

# setup the column names for today's data from the nseindia.com
header_new = ['ClosePrice'+today, 'TotalTradedQuantity'+today, 'TurnoverInLacs'+today, 'DeliveryQuantity'+today, 'DeliveryPercentage'+today]
header_new


# extract the new data and paste to the main bhav data file.
for i in range(5):
    new_head = header_new[i]
    prev_head = main_bhav_data.columns.get_loc(header_main[i]) + 1

    extracted_col = today_bhav_data[new_head]
    main_bhav_data.insert(prev_head, new_head, extracted_col)

In [63]:
main_bhav_data

,Symbol,is_valid,Series,ClosePrice10102022,ClosePrice11102022,ClosePrice12102022,ClosePrice13102022,ClosePrice14102022,TotalTradedQuantity10102022,TotalTradedQuantity11102022,...,DeliveryQuantity10102022,DeliveryQuantity11102022,DeliveryQuantity12102022,DeliveryQuantity13102022,DeliveryQuantity14102022,DeliveryPercentage10102022,DeliveryPercentage11102022,DeliveryPercentage12102022,DeliveryPercentage13102022,DeliveryPercentage14102022
0,ADANIPORTS,1,EQ,808.10,797.35,803.90,784.75,780.10,4642360,6193388,...,997903,1304344,940688,1510646,650375,21.50,21.06,16.71,20.16,14.25
1,APOLLOHOSP,1,EQ,4340.60,4275.55,4287.05,4266.05,4297.15,207878,258216,...,38732,106718,414814,134839,94375,18.63,41.33,66.76,48.67,37.10
2,ASIANPAINT,1,EQ,3277.95,3298.70,3248.20,3209.00,3185.50,734377,1411316,...,302946,523181,730138,356092,511708,41.25,37.07,40.28,45.66,60.50
3,AXISBANK,1,EQ,776.85,785.55,808.70,802.00,800.50,14106654,15953495,...,8697279,8916019,14015974,7174692,3810137,61.65,55.89,66.03,55.08,46.35
4,BAJAJ-AUTO,1,EQ,3616.95,3528.75,3624.50,3604.80,3570.50,240975,258827,...,119562,128336,146509,175361,256918,49.62,49.58,29.52,52.10,51.39
5,BAJAJFINSV,1,EQ,1684.40,1672.65,1694.65,1681.75,1688.30,1229639,1402150,...,313142,439030,477717,248944,387881,25.47,31.31,31.04,23.37,31.30
6,BAJFINANCE,1,EQ,7293.40,7202.80,7226.20,7151.65,7272.50,653113,732206,...,139108,239519,221445,157860,267321,21.30,32.71,29.90,25.94,31.86
7,BHARTIARTL,1,EQ,792.35,780.30,775.55,768.90,765.95,3519416,3699862,...,2247568,2047994,4005208,1519596,2176149,63.86,55.35,66.93,53.47,60.31
8,BPCL,1,EQ,304.65,299.75,302.50,300.30,297.85,2365616,2652669,...,824116,1425085,2450766,1334588,1166818,34.84,53.72,33.14,44.06,45.27
9,BRITANNIA,1,EQ,3797.85,3727.90,3763.00,3784.60,3796.25,185199,202644,...,108465,118939,199778,94118,253321,58.57,58.69,73.68,60.04,72.35


In [64]:
main_bhav_data.to_csv('nse_data_file.csv', index=False)